In [1]:
import sys
import string
import nltk as nltk
from nltk.tokenize import RegexpTokenizer,sent_tokenize,word_tokenize
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
import os
from sklearn.metrics import confusion_matrix,classification_report
import matplotlib.pyplot as plt
import pandas  as pd
import json
import numpy as np
from gensim import corpora
import gensim
from sklearn import preprocessing
from sklearn.naive_bayes import BernoulliNB
import json
from sklearn.naive_bayes import MultinomialNB

In [2]:
busfile ='/home/rungsunan/data/yelp/business.json'
reviewfile = '/home/rungsunan/data/yelp/review.json'

reviewfilecopy = '/home/rungsunan/data/yelp/reviewcopy.json'
reviewfiledir = '/home/rungsunan/data/yelp/'

In [14]:
business_df = pd.read_json(busfile,lines=True)
business_df.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,None,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
3,"15655 W Roosevelt St, Ste 237",None,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,85338,3,5.0,AZ
4,"4209 Stuart Andrew Blvd, Ste F","{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...",HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",1,35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC


The reviewfile is too big to import with pd.read_json so we use a for loop to process iteratively with a dictionary comprehension. We are only interested in the review text and business id, so we only parse those fields from the json file in order to save space. Since the json file is newline separated, it is better handled with the json.loads() method. 

In [4]:
revdictlist = []
with open('/home/rungsunan/data/yelp/review.json','r') as f1:
    for i,line in enumerate(f1,1):
      revdictlist.append({ k: v for k, v in json.loads(line).items() if k.startswith('business_id') or k.startswith('text')})

In [5]:
len(revdictlist)

6685900

In [6]:
revdictlist[0]

{'business_id': 'ujmEBvifdJM6h6RLv4wQIg',
 'text': 'Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.'}

Now we have a list of dictionaries containing the business_id and reviewtext of each review. We collapse this list into a single dictionary which has the row number as the key, and the business_id and review text as the fields. 

In [7]:
reviewdict={}
for i,val in enumerate(revdictlist):
    reviewdict[i] = val

In [8]:
reviewdict[1]

{'business_id': 'NZnhc2sEQy3RmzKTZnqtwQ',
 'text': "I *adore* Travis at the Hard Rock's new Kelly Cardenas Salon!  I'm always a fan of a great blowout and no stranger to the chains that offer this service; however, Travis has taken the flawless blowout to a whole new level!  \n\nTravis's greets you with his perfectly green swoosh in his otherwise perfectly styled black hair and a Vegas-worthy rockstar outfit.  Next comes the most relaxing and incredible shampoo -- where you get a full head message that could cure even the very worst migraine in minutes --- and the scented shampoo room.  Travis has freakishly strong fingers (in a good way) and use the perfect amount of pressure.  That was superb!  Then starts the glorious blowout... where not one, not two, but THREE people were involved in doing the best round-brush action my hair has ever seen.  The team of stylists clearly gets along extremely well, as it's evident from the way they talk to and help one another that it's really genuin

We convert the dictionary of reviews to a dataframe with the columns business_id and text

In [9]:
reviews_df = pd.DataFrame.from_dict(reviewdict,orient='index',columns=['business_id','text'])
reviews_df.head()

,business_id,text
0,ujmEBvifdJM6h6RLv4wQIg,Total bill for this horrible service? Over $8G...
1,NZnhc2sEQy3RmzKTZnqtwQ,I *adore* Travis at the Hard Rock's new Kelly ...
2,WTqjgwHlXbSFevF32_DJVw,I have to say that this office really has it t...
3,ikCg8xy5JIg_NGPx-MSIDA,Went in for a lunch. Steak sandwich was delici...
4,b1b1eb3uo-w561D0ZfCEiQ,Today was my second out of three sessions I ha...


We are making a dataset with just review text of Chinese or Italian restaurants. From the business dataframe we take the businesses with those types in the categories column. From the review list we select the business_id and the review text. Then merge the two dataframes to create a training/test set. 

In [18]:
business_df = business_df[business_df['categories'].str.contains("Chinese|Italian")==True]
business_df = business_df[['business_id','categories']]
business_df.loc[business_df['categories'].str.contains("Chinese"), "cuisine"] = "Chinese"
business_df.loc[business_df['categories'].str.contains("Italian"), "cuisine"] = "Italian"
business_df = business_df[['business_id','cuisine']]
business_df.head()

,business_id,cuisine
1,QXAEGFB4oINsVuTFxEYKFQ,Chinese
13,fweCYi8FmbJXHCqLnwuk8w,Italian
17,PZ-LZzSlhSe9utkQYU8pFg,Italian
35,BvYU3jvGd0TJ7IyZdfiN2Q,Italian
45,e_EMySqP0uwlVZfd8mRaaQ,Chinese


In [19]:
print('Number of Italian or Chinese Restaurants: ' + 
      str(len((business_df[(business_df.cuisine == 'Italian') | (business_df.cuisine == 'Chinese')]))))
print('Number of Italian Restaurants: ' + str(len((business_df[(business_df.cuisine == 'Italian')]))))
print('Number of Chinese Restaurants: ' + str(len((business_df[(business_df.cuisine == 'Chinese')]))))

Number of Italian or Chinese Restaurants: 9349
Number of Italian Restaurants: 4716
Number of Chinese Restaurants: 4633


We only are interested in the businessID and the text of the reviews

In [20]:
reviews_df = reviews_df[['business_id','text']]
reviews_df.head()

,business_id,text
0,ujmEBvifdJM6h6RLv4wQIg,Total bill for this horrible service? Over $8G...
1,NZnhc2sEQy3RmzKTZnqtwQ,I *adore* Travis at the Hard Rock's new Kelly ...
2,WTqjgwHlXbSFevF32_DJVw,I have to say that this office really has it t...
3,ikCg8xy5JIg_NGPx-MSIDA,Went in for a lunch. Steak sandwich was delici...
4,b1b1eb3uo-w561D0ZfCEiQ,Today was my second out of three sessions I ha...


Finally, we can now merge the two dataframes into one data frame containing the business_id, review_text and cuisine code. 

In [22]:
final_df =  pd.merge(business_df,reviews_df,on='business_id')
final_df.head(1000)

,business_id,cuisine,text
0,QXAEGFB4oINsVuTFxEYKFQ,Chinese,My girlfriend and I went for dinner at Emerald...
1,QXAEGFB4oINsVuTFxEYKFQ,Chinese,We've always been there on a Sunday so we were...
2,QXAEGFB4oINsVuTFxEYKFQ,Chinese,"***No automatic doors, not baby friendly!*** I..."
3,QXAEGFB4oINsVuTFxEYKFQ,Chinese,"Horrible service,\nI went there tonight with m..."
4,QXAEGFB4oINsVuTFxEYKFQ,Chinese,One of the gauges of a good Chinese restaurant...
5,QXAEGFB4oINsVuTFxEYKFQ,Chinese,"I've been a frequent at this place for years, ..."
6,QXAEGFB4oINsVuTFxEYKFQ,Chinese,We chose this restaurant for our Chinese New Y...
7,QXAEGFB4oINsVuTFxEYKFQ,Chinese,I went at 230 on a Monday. It was dimsum \n\nI...
8,QXAEGFB4oINsVuTFxEYKFQ,Chinese,My family and I were at Emerald yesterday duri...
9,QXAEGFB4oINsVuTFxEYKFQ,Chinese,"Because we are in Mississauga, I think this pl..."


In [23]:
print('Number of Italian Restaurant reviews: ' + str(len((final_df[(final_df.cuisine == 'Italian')]))))
print('Number of Chinese Restaurant reviews: ' + str(len((final_df[(final_df.cuisine == 'Chinese')]))))

Number of Italian Restaurant reviews: 392125
Number of Chinese Restaurant reviews: 262695


For text analysis we use a code for the cuisine type(-1 for italian, 1 for chinese), and make all of the text lowercase for stemming.

In [24]:
final_df["cuisine_code"] = np.where(final_df["cuisine"].str.contains("Italian"), 1,-1)
final_df['text'] = final_df['text'].str.lower()

# Exploratory

In [25]:
final_df['length'] = final_df['text'].apply(len)
final_df.length.describe()

count    654820.000000
mean        597.582896
std         560.578110
min           1.000000
25%         232.000000
50%         420.000000
75%         765.000000
max        5000.000000
Name: length, dtype: float64

In [26]:
final_df.head()

,business_id,cuisine,text,cuisine_code,length
0,QXAEGFB4oINsVuTFxEYKFQ,Chinese,my girlfriend and i went for dinner at emerald...,-1,1475
1,QXAEGFB4oINsVuTFxEYKFQ,Chinese,we've always been there on a sunday so we were...,-1,439
2,QXAEGFB4oINsVuTFxEYKFQ,Chinese,"***no automatic doors, not baby friendly!*** i...",-1,252
3,QXAEGFB4oINsVuTFxEYKFQ,Chinese,"horrible service,\ni went there tonight with m...",-1,1113
4,QXAEGFB4oINsVuTFxEYKFQ,Chinese,one of the gauges of a good chinese restaurant...,-1,1668


In [ ]:
stop_words = get_stop_words("english")
print(stop_words)
len(stop_words)

In [ ]:
outcome_reviews = list(zip(final_df.cuisine_code,final_df.text))
print("Total number of Chinese/Italian reviews: " + str(len(outcome_reviews)))
outcome_reviews[0]

let's put the text and the cuisine code into a set(reviewtext, cuisinecode)

In [ ]:
outcomeslist, reviewslist = list(zip(*outcome_reviews))

In [ ]:
outcomeslist[0],reviewslist[0]

In [ ]:
tokenizer = RegexpTokenizer(r'\s+', gaps=True)

# create English stop words list
en_stop = set(get_stop_words('en'))
#print(en_stop)
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

In [ ]:
punctuation_string = '\"?!@#$%^&*()\';:+,/.-|~=\\'
table = str.maketrans(dict.fromkeys(punctuation_string))

In [ ]:
corpus = []
for review in reviewslist:
    raw = review.translate(table)
    tokens = tokenizer.tokenize(raw)
    # remove stop words from tokens
    stopped_tokens = [j for j in tokens if not j in en_stop]
    # print(tokens)
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(k) for k in stopped_tokens]
    corpus.append(stemmed_tokens)

In [ ]:
corpus[0]

In [ ]:
dictionary = gensim.corpora.Dictionary(corpus)

In [ ]:
lexicon = [dictionary.doc2bow(doc) for doc in corpus ] 

In [ ]:
outcomeslist[0]

In [ ]:
tfidf = gensim.models.TfidfModel(dictionary=dictionary,normalize=True)

In [ ]:
vectors = [tfidf[id2word.doc2bow(doc)] for doc in corpus ] 

In [ ]:
vectors[2]

In [ ]:
corpus_bow = [id2word.doc2bow(doc) for doc in corpus ]

In [ ]:
corpus_bow[0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(list(final_df.text))

In [ ]:
count_vect.vocabulary_

In [ ]:
rev4= reviewslist[4]

In [ ]:
bow4 = count_vect.transform([rev4])
print(bow4)
print(bow4.shape)

In [ ]:
print(count_vect.get_feature_names()[90274])

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)

In [ ]:
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

In [ ]:
clf = MultinomialNB().fit(X_train_tfidf, outcomeslist)

In [ ]:
X_test[0]

In [ ]:
X_test_counts = count_vect.fit_transform(X_test)

In [ ]:
X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)
X_test_tfidf.shape

In [ ]:
a= clf.predict(X_train_counts)

In [ ]:
a

In [ ]:
print(accuracy_score(outcomeslist,a))
print(confusion_matrix(outcomeslist,a))

In [ ]:
train_data_features = vectorizer.fit_transform(corpus_bow)

In [ ]:

plt.matshow(confusion_matrix(outcomeslist, a), cmap=plt.cm.binary, interpolation='nearest')
plt.title('confusion matrix')
plt.colorbar()
plt.ylabel('expected label')
plt.xlabel('predicted label')

In [ ]:
print(classification_report(outcomeslist,a))

In [ ]:
final_df.length.describe()

In [ ]:
final_df.length.plot(bins=20, kind='hist')

In [ ]:
final_df.hist(column='length', bins=50)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
?train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(list(final_df.text),outcomeslist, test_size=0.33, random_state=42)

print(len(X_train),len(X_test),len(X_train)+len(X_test))

In [ ]:
len(X_train)

In [ ]:

%time cuisine_detector = MultinomialNB().fit(X_train_tfidf,outcomeslist)

In [ ]:
print ('predicted:', cuisine_detector.predict(bow4)[0])
print ('expected:', outcomeslist[3])

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [ ]:
scores = cross_val_score(pipeline,  # steps to convert raw messages into models
                         X_train,  # training data
                         y_train,  # training labels
                         cv=10,  # split data randomly into 10 parts: 9 for training, 1 for scoring
                         scoring='accuracy' # which scoring metric?
                         
                         )
print (scores)

In [ ]:
print( scores.mean(), scores.std())

In [ ]:
filecounter=0
array = []
with open(reviewfile, "r") as f:
  for i,line in enumerate(f,1):
    array.append(line)
    if ((i % 500000) == 0):
      filecounter += 1
      reviewfilename = reviewfiledir + 'reviewfile_' + str(filecounter) + '.json'
      with open(reviewfilename, "w") as x:
            x.write(''.join(array))
      array = []
f.close()

In [ ]:
with open('/home/rungsunan/data/yelp/reviewtext.json', "w") as x:
            json.dump(reviewdict,x)

In [ ]:
filelist = [x for x in list(os.listdir('/home/rungsunan/data/yelp')) if x[:10] == 'reviewfile']

In [ ]:
li = []
for filename in filelist:
    df = pd.read_json(reviewfiledir + filename,lines=True)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
json.loads(reviewfiledir+'reviewfile_1.json')

In [ ]:
with open(reviewfiledir + 'reviewfile_1.json',"r") as f:
    for i,line in enumerate(f):
        testdict = json.loads(line)